Load required libraries.

In [63]:
import pickle
import numpy as np
from itertools import combinations_with_replacement, product
from copy import deepcopy

import Yahtzee  # LETS GO :))))

This program creates the model for Yahtzee game.


Ignoring the `Bonus`, simplified (actual scores aren't remembered), there are a total of:
- 3 rerolls (2 bits)
- 2^13 ways to fill categories (13 bits)
- 10C5 unique dice combinations (8 bits)
For a total of 6,193,152 states.


These states has been discretized and represented with 23 bits:
- First 2 bits represent number of rerolls left (0~2)
- Next 13 bits represent whether categories are filled
- Last 8 bits represent the IDs of unique dice combinations.


The IDs of each unique dice combination is simply the index of the combinations list.
Fortunately, these are already all sorted in ascending order of die faces.
Mapping between them is done with dictionaries.

In [64]:
dice_combinations = list(combinations_with_replacement(range(1, 7), 5))
num_of_combinations = len(dice_combinations)  # 252

id_to_combination = {}
combination_to_id = {}
for id in range(num_of_combinations):
    id_to_combination[id] = dice_combinations[id]
    combination_to_id[dice_combinations[id]] = id

There are total of 2^5 = 32 ways of selecting dice to reroll.
Since (0,0,0,0,0), or selecting no die for reroll is illegal, it is not accounted for.

This will also be used for action space later. Simplest way to generate them is buy converting integers to bits.

In [65]:
num_of_selections = 32
selections = []
for i in range(1, 32):
    select_combination = (i >> 4, i >> 3 & 1, i >> 2 & 1, i >> 1 & 1, i & 1)
    selections.append(select_combination)

Now it is time for transition functions.


Each dice rolled may lead to 6 other states for dice, exponentially increasing per dice rerolled.
That means, for every state, there are 31 reroll combinations and each combination leads to
6^number of selected dice combinations.

The resulting combination however is not unique to selection combination, for example:
- Current dice: 1,1,2,2,3
  - Reroll: 1,0,1,0,0, Result: 4,1,4,2,3 -> 1,2,3,4,4                
  - Reroll: 0,1,0,1,0, Result: 1,4,2,4,3 -> 1,2,3,4,4

This is why the probability of reaching a certain combination is not simply just (1/6)^dice rolled.
There may be some clever math to work it out, but we have computer, memory and time, (most importantly, I'm too stupid)
so why not just count the whole thing and then compute the probability?

In [39]:
dice_neighbours = np.empty(252, dtype=object)

for combination in dice_combinations:  # total of 252 combinations

    # print("Currently looking at: ", combination)

    transitions = []

    for selection in selections:  # total of 31 ways to select

        new_combinations = {}

        # Keep the kept faces.
        kept_faces = []
        for i in range(5):
            if selection[i] == 0:
                kept_faces.append(combination[i])
        
        # Generate all new combinations.
        num_of_selected_dice = sum(selection)
        new_faces = list(product(range(1, 7), repeat=num_of_selected_dice))
        total = len(new_faces)
        for new_face in new_faces:
            new_combination = sorted(kept_faces + list(new_face))
            try:
                new_combinations[tuple(new_combination)] += (1 / total)
            except KeyError:
                new_combinations[tuple(new_combination)] = (1 / total)
        
        # Converting to list for easier loop operations in future.
        transition = []
        for key in new_combinations.keys():
            pair = (key, new_combinations[key])
            transition.append(pair)
        
        transitions.append((selection, transition))

        # print("Action completed for: ", selection)
    
    id = combination_to_id[combination]
    dice_neighbours[id] = transitions

np.save("dice_neighbours.npy", dice_neighbours)

In [40]:
print(dice_neighbours[0][30])
len(dice_neighbours[0][30][1])

((1, 1, 1, 1, 1), [((1, 1, 1, 1, 1), 0.0001286008230452675), ((1, 1, 1, 1, 2), 0.0006430041152263376), ((1, 1, 1, 1, 3), 0.0006430041152263376), ((1, 1, 1, 1, 4), 0.0006430041152263376), ((1, 1, 1, 1, 5), 0.0006430041152263376), ((1, 1, 1, 1, 6), 0.0006430041152263376), ((1, 1, 1, 2, 2), 0.0012860082304526753), ((1, 1, 1, 2, 3), 0.00257201646090535), ((1, 1, 1, 2, 4), 0.00257201646090535), ((1, 1, 1, 2, 5), 0.00257201646090535), ((1, 1, 1, 2, 6), 0.00257201646090535), ((1, 1, 1, 3, 3), 0.0012860082304526753), ((1, 1, 1, 3, 4), 0.00257201646090535), ((1, 1, 1, 3, 5), 0.00257201646090535), ((1, 1, 1, 3, 6), 0.00257201646090535), ((1, 1, 1, 4, 4), 0.0012860082304526753), ((1, 1, 1, 4, 5), 0.00257201646090535), ((1, 1, 1, 4, 6), 0.00257201646090535), ((1, 1, 1, 5, 5), 0.0012860082304526753), ((1, 1, 1, 5, 6), 0.00257201646090535), ((1, 1, 1, 6, 6), 0.0012860082304526753), ((1, 1, 2, 2, 2), 0.0012860082304526753), ((1, 1, 2, 2, 3), 0.003858024691358023), ((1, 1, 2, 2, 4), 0.0038580246913580

252

Now define the action space.

The possible actions in each space are:
- Write in to one of the category (13 options)
- Select dices to roll (31 ways)

For a total of 44 actions.

They are indexed from 0~43, where 0~30 are the dice combinations, 31~43 are selecting each of the 13 categories in ascending order, where 31 is writing in to `Ones`, 32 to `Twos`, ..., 43 to `Chance`. For reference, the categories are:
- 32: Ones
- 33: Twos
- 34: Threes
- 35: Fours
- 36: Fives
- 37: Sixes
- 38: Three-of-a-Kind
- 39: Four-of-a-Kind
- 40: Full House
- 41: Small Straight
- 42: Large Straight
- 43: Yahtzee
- 44: Chance

In [66]:
num_of_actions = 44
category_names = ["Ones", "Twos", "Threes", "Fours", "Fives", "Sixes", "Three-of-a-Kind", "Four-of-a-Kind", "Full House", "Small Straight", "Large Straight", "Yahtzee", "Chance"]
id_to_action = {}
action_to_id = {}
for id in range(31):
    id_to_action[id] = selections[id]
    action_to_id[selections[id]] = id
for id in range(13):
    id_to_action[id + 31] = category_names[id]
    action_to_id[category_names[id]] = id

In [67]:
# Test
print(id_to_action[20], id_to_action[43])
print(action_to_id[(1,1,0,0,1)], action_to_id["Three-of-a-Kind"])

(1, 0, 1, 0, 1) Chance
24 6


State space is defined as above, but defining 6,193,152 states with 23 bits (8,388,608) means at least 26% of states are empty and not in use.

To save time during iteration algorithm and also condense the space, split the index to many sub partitions:
- 3 rerolls [0~2,064,383], [2,064,384~4,128,767], [4,128,768~6,193,151]
    - 8,192 categories [[0~251, 252~503, ...]], [[2,064,384~2,064,635, 2,064,636~2,064,888, ... ]], ...
        - 252 dice combinations [[[0], [1], ...]], [[[2,064,384], [2,064,385], ...]], ...

Write a parser for index and encoder for states.

In [68]:
# Bits are read from left to right.
def parse_state(index):
    rerolls_left, temp = divmod(index, 2064384)  # [0 ~ 2]
    categories, dice_combination = divmod(temp, 252)
    return rerolls_left, categories, dice_combination


def get_state_id(state):
    rerolls_left, categories, dice_combination = state
    id = rerolls_left * 2064384 + categories * 252 + dice_combination
    return id

In [70]:
print(parse_state(5160960))
print(id_to_category[4096])
print(id_to_combination[0])
print(get_state_id((0, 16, 214)))

(2, 4096, 0)
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(1, 1, 1, 1, 1)
4246


Total of 2^13 possible combination of categories, index them similar to the dice selections.

In [45]:
category_combinations = []
for i in range(0, 8192):
    category = []
    for j in reversed(range(13)):
        category.append(i >> j & 1)
    category_combinations.append(category)

id_to_category = {}
category_to_id = {}
for id in range(8192):
    id_to_category[id] = category_combinations[id]
    category_to_id[tuple(category_combinations[id])] = id

In [46]:
# Test
print(category_to_id[(1,0,0,0,1,1,0,0,1,1,1,0,1)])
print(id_to_category[4246])

4509
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]


Next we define the reward function. Reward will be simply the score obtained from writing into the category. Since this is invariant of successor state, it is in the form of R(s, a)

May also give negative reward for giving up on current dice combination.

In [59]:
neg_multiplier = 0
pos_multiplier = 1

def calc_reward(state, action):
    reward = 0
    rerolls_left, categories, dice_combination = parse_state(state)
    if action <= 30:  # rolling dice
        pass
    else:  # writing in to category 
        category_index = action - 31
        parsed_dice = id_to_combination[dice_combination]
        score = Yahtzee.CATEGORIES_SCORING[category_index](np.array(parsed_dice))
        reward += score * pos_multiplier
    return reward

Now we finally model the game. For the total of 6,193,152 states, we create a transition function for each action: 
- For actions that roll the dice, it should transition to state where reroll count is reduced, categories are kept with new dice combination.
- For actions that write the category, it should transition to state with that category written, reroll reset with fresh dice combination.
- For illegal actions (rolling when no rerolls, writing to filled category), it will not be available in the state.


In [48]:
# Action check to see if it is legal.
def is_legal_action(state, action):
    rerolls_left, categories, dice_combination = parse_state(state)
    if action <= 30:  # rolling dice
        return rerolls_left > 0
    else:  # writing in to category
        category_index = action - 31
        return categories[category_index] == 0

In [ ]:
# THE MODEL
yahtzee_model = np.empty(6193152, dtype=object)
num_of_states = 6193152

for state in range(num_of_states):
    state_model = []
    rerolls_left, categories, dice_combination = parse_state(state)
    
    # Model all actions to successor states with probability and reward.
    for action in range(num_of_actions):

        if action <= 30:  # rerolling dice
            if rerolls_left > 0:  # are there rerolls left?
                state_transition = []
                new_rerolls_left = rerolls_left - 1
                curr_dice_id = combination_to_id[dice_combination]
                action_transition = dice_neighbours[curr_dice_id][action]
                reward = calc_reward(state, action)  # reward is independent of successor so compute outside to save calculations
                # Now we need to append the rerolls_left, categories and dice_combination for proper state number.
                for next_combination in action_transition[1]:
                    next_combination_id = combination_to_id[next_combination[0]]
                    next_state_id = get_state_id((new_rerolls_left, categories, next_combination_id))
                    next_state_prob = next_combination[1]
                    state_transition.append((next_state_id, next_state_prob, reward))
                state_model.append((action, state_transition))

        else:  # writing in to category
            category_index = action - 31
            parsed_categories = id_to_category[categories]
            if parsed_categories[category_index] == 0:  # is category available?
                state_transition = []
                new_rerolls_left = 2
                # Updating category.
                new_category = list(parsed_categories)
                new_category[category_index] = 1
                # print(new_category)
                new_category = tuple(new_category)
                # Get the id of category to encode to state.
                new_category = category_to_id[new_category]
                # print(categories, parsed_categories, new_category)
                action_transition = dice_neighbours[0][30]  # reroll everything
                reward = calc_reward(state, action)
                # Now we need to append the rerolls_left, categories and dice_combination for proper state number.
                for next_combination in action_transition[1]:
                    next_combination_id = combination_to_id[next_combination[0]]
                    next_state_id = get_state_id((new_rerolls_left, new_category, next_combination_id))
                    next_state_prob = next_combination[1]
                    state_transition.append((next_state_id, next_state_prob, reward))
                state_model.append((action, state_transition))
    yahtzee_model[state] = state_model
    print("Finished state: ", state)
    
np.save("yahtzee_model.npy", yahtzee_model)